<a href="https://www.kaggle.com/code/dsptlp/a-b-testing-stargazer-python?scriptVersionId=213234542" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from IPython.display import HTML, display

url = "https://github.com/pparkitn/kaggle/blob/main/Images/A_B_Testing_Dino.png?raw=true"
display(HTML(f'<img src="{url}" style="max-width:100%;">'))

# Analyzing A/B Test
Super Simple Notebook for analyzing A/B Test Results in Python
- Covariate Balance Check
- Regression Model Result
- Heterogeneous Treatment Effect


In [2]:
!pip install stargazer

import numpy as np
from datetime import datetime
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

input_file = '/kaggle/input/ab-test-linkedin-attractiveness-bias/1_FINAL_MASTER_TRACK.xlsx'

In [3]:
filtered_data = pd.read_csv('/kaggle/input/ab-test-linkedin-attractiveness-bias/test_data.csv')
filtered_data.head(5)

,Source,job_title_grp,gender,gender_corr,treatment,group,connected,complia
0,LU,manager,m,m,1,A-W,0,1
1,LU,executive,m,m,1,A-W,1,1
2,LU,executive,NaN,m,1,A-W,1,1
3,LU,consultant,f,f,1,A-W,0,1
4,LU,consultant,m,m,1,A-W,0,1


# Covariate Balance Check- Rregression of our collected covariates VS treatment
-  The result shows that although we have small differences between control and treatment (U-W VS A-W) the differences are very small and not statistically significant
-  The covariance balance check passes, we will use these covariates in our final results regression model but they should not impact our treatment effect as they are balanced.


Checking covariate balance is essential for verifying the integrity of randomization. Strong coverage might include:

- Comparison of Groups: Use summary statistics (means, medians, standard deviations) and visualizations (e.g., histograms, boxplots) to compare treatment and control groups.

- Statistical Tests: Conduct tests (e.g., t-tests, chi-square tests, or Kolmogorov-Smirnov tests) for balance on key covariates.
- Adjustment if Needed: If imbalances exist, discuss potential remedies, such as using stratified analyses or including covariates in the regression model.

In [4]:
# Model 1
model_1 = smf.ols('treatment ~ gender_corr', data=filtered_data).fit()

# Model 2
model_2 = smf.ols('treatment ~ gender_corr + Source', data=filtered_data).fit()

# Model 3
model_3 = smf.ols('treatment ~ gender_corr + Source + job_title_grp', data=filtered_data).fit()

# Collecting robust standard errors
robust_se_1 = model_1.get_robustcov_results().bse
robust_se_2 = model_2.get_robustcov_results().bse
robust_se_3 = model_3.get_robustcov_results().bse

stargazer = Stargazer([model_1, model_2, model_3])
stargazer.title("Regression Table - Covariance Balance Check")
stargazer.custom_columns(["Simple-Model", "Regular-Model", "Full-Model"], [1, 1, 1])
stargazer.add_line("Other Covariate", ["No", "No", "job-title-grp"])
stargazer.show_r2 = False
stargazer.show_adj_r2 = False
stargazer.show_f_statistic = False
stargazer.show_degrees_of_freedom = False

stargazer

# Regression Model

- The regression model will be used to estimate the treatment effect along with statistical significance. We use heteroskedasticity-robust standard errors when reporting all regression results in the paper.

- The four models we analyze in the regression table. We considered running a test to determine which model is the best however all the models produce a similar treatment effect along with standard errors which renders the test not required. We select model #3 to use to report our findings. 
- 
The coefficient of receiving the treatment is `r coef(model_3)[[2]]` with a p-value of `r coef(summary(model_3))[, "Pr(>|t|)"][2]`. We achieve statistical significance with our p-value < 0.05. Null hypothesis is rejected, we accept the alternative hypothesis that attractiveness plays a role in accepting a new connection on LinkedIn.

- All models treatment coefficient remains relatively constant when adding covariates. The near zero change would indicate that the blocking that was created on gender/source and job-title-grp was executed correctly. 

- Interesting observation is that the gender of the subject played a role in connection acceptance. Based on regression results male subject was 0.097 times more willing to accept connection with a p-value < 0.01.

- Observations from Lucas had a coefficient of 0.1 with a p-value < 0.05. As the source variable can be used as a proxy for the country and Lucas connections would be the proxy for Brazil we can make a claim that people from Brazil are impacted by the treatment more than the USA and Canada.  

- As the connection counter grows, we do get a better acceptance rate which can be seen as ccon has a coefficient of 0.0002 with a p-value < 0.1. 

# Regression Model Results
Regression analysis is a powerful way to quantify treatment effects and account for covariates. Ensure this section covers:

- Model Specification: Clearly outline the model being used (e.g., linear regression, logistic regression).
- Main Effects: Report the treatment effect estimate (𝛽β) and its statistical significance (p-values, confidence intervals).
- Model Diagnostics: Include checks for assumptions (e.g., linearity, homoscedasticity, multicollinearity).

In [5]:
# Define the models
model_1 = smf.ols('connected ~ treatment ', data=filtered_data).fit()
model_2 = smf.ols('connected ~ treatment + Source', data=filtered_data).fit()
model_3 = smf.ols('connected ~ treatment + Source + gender_corr', data=filtered_data).fit()
model_4 = smf.ols('connected ~ treatment + Source + gender_corr + job_title_grp', data=filtered_data).fit()

# Collect robust standard errors
robust_se_1 = model_1.get_robustcov_results().bse
robust_se_2 = model_2.get_robustcov_results().bse
robust_se_3 = model_3.get_robustcov_results().bse
robust_se_4 = model_4.get_robustcov_results().bse

# Generate a regression table using Stargazer
stargazer = Stargazer([model_1, model_2, model_3, model_4])
stargazer.title("Regression Table - Final Result")
stargazer.custom_columns(["Model 1", "Model 2", "Model 3", "Model 4"], [1, 1, 1, 1])
stargazer.add_line("Other Covariate", ["No", "No", "No", "job-title-grp"])
#stargazer.omit_covariates("job_title_grp")
stargazer.show_r2 = False
stargazer.show_adj_r2 = False
stargazer.show_f_statistic = False
stargazer.show_degrees_of_freedom = False

stargazer

# Heterogeneous Treatment Effect (HTE)
A Heterogeneous Treatment Effect (HTE) refers to the variation in the effect of a treatment, intervention, or policy across different subgroups or individuals within a population. In essence, it acknowledges that "one size does not fit all" and that the impact of a treatment can differ based on individual characteristics or contextual factors.

This is a critical section, as understanding treatment effect variation can unlock insights beyond the average treatment effect (ATE). To strengthen it:

- Identify Subgroups: Explicitly state how you define subgroups (e.g., by demographics, behavior).
- Methods: Detail the methodology for estimating HTEs (e.g., interaction terms in regression, causal forests, or other machine learning techniques).
- Visualization: Use visuals like forest plots or conditional effect plots to communicate variation effectively.
- Significance Testing: Evaluate the robustness of detected heterogeneity (e.g., multiple hypothesis testing corrections).


- Analyze the heterogeneous treatment effect across all the collected covariates using a regression model indicator variable. The results do not have any results with a statistically significant result.    


In [6]:
# Define the models
model_1 = smf.ols('connected ~ treatment + gender_corr + Source + treatment*gender_corr + treatment*Source', data=filtered_data).fit()

# Collect robust standard errors
robust_se_1 = model_1.get_robustcov_results().bse

# Generate a regression table using Stargazer
stargazer = Stargazer([model_1])
stargazer.title("Regression Table - HTE")
stargazer.custom_columns(["Model 1"], [1])
stargazer.add_line("Other Covariate", ["No"])
stargazer.show_r2 = False
stargazer.show_adj_r2 = False
stargazer.show_f_statistic = False
stargazer.show_degrees_of_freedom = False

stargazer